## [무작정 kaggle 따라하기] Interactive Porto Insights (Plot.ly)

*kaggle 연습을 위한 notebook입니다.* <br>
해당 커널은 다음 자료를 참고하였습니다.

- [Anisotropic - Interactive Porto Insights - A Plot.ly Tutorial](https://www.kaggle.com/arthurtok/interactive-porto-insights-a-plot-ly-tutorial)

## Introduction

해당 notebook은 브라질에서 세번째로 큰 보험회사 [Porto Seguro](https://en.wikipedia.org/wiki/Porto_Seguro_S.A.)의 예측 문제(운전자가 내년에 보험을 가동할 가능성)을 담고 있다.

이 notebook은 Python의 시각화 라이브러리 Plot.ly를 이용하여 해당 데이터를 시각화하고 인사이트를 도출하는데 의미를 두고 있다. Plot.ly는 웹 온라인 상에서 그래픽, 통계학 시각화를 제공하는데 특화된 소프트웨어 회사로써 Python, R, Matlb, Node.js 등 다양한 프로그래밍 언어에 접근이 가능하게 API를 제공한다.

**해당 노트북에서 보여줄 Plot.ly 차트는 다음과 같다.**

- 일반 수평 bar plot (타겟변수분포를 조사하는데 사용함)

- 상관관계 heatmap plot

- 산점도 scatter plot

- 수직 bar plot (내림차순으로 정렬, 다양한 변수들의 중요성 따짐)

- 3D scatter plot

이 notebook은 다음과 같이 이뤄진다.

- 데이터 품질 검사 (시각화 및 결측치 값 처리)

- 변수 검사와 필터링 (상관관계와 타겟 변수와 관련된 상호정보량 알아보기. 이진, 범주형 및 다른 변수들 조사)

- 머신러닝 모델을 통한 변수 중요도 순위 매기기 (Random Forest와 Gradient Boost 이용함)

In [ ]:
# setting
import pandas as pd
import numpy as np
import seaborn as sns
import matplotlib.pyplot as plt
%matplotlib inline
import plotly.offline as py
py.init_notebook_mode(connected=True)
import plotly.graph_objs as go
import plotly.tools as tls
import warnings
from collections import Counter
from sklearn.feature_selection import mutual_info_classif
warnings.filterwarnings('ignore')

# Input data files are available in the "../input/" directory.
# For example, running this (by clicking run or pressing Shift+Enter) will list all files under the input directory

import os
for dirname, _, filenames in os.walk('/kaggle/input'):
    for filename in filenames:
        print(os.path.join(dirname, filename))

# Any results you write to the current directory are saved as output.

In [ ]:
# load the training data
train = pd.read_csv('/kaggle/input/porto-seguro-safe-driver-prediction/train.csv')
train.head()

In [ ]:
rows = train.shape[0]
columns = train.shape[1]
print('The train dataset contains {0} rows and {1} columns'.format(rows, columns))

---

## 1. 데이터 품질 검사 Data Quality Checks

### 결측치 검사하기

In [ ]:
train.isnull().any().any()

여기서 False가 나오니 null값이 없다고 볼 수 있다. [하지만 '-1' 값이 들어있으면 해당 변수를 결측치로 본다고 정보란에 적혀 있다.](https://www.kaggle.com/c/porto-seguro-safe-driver-prediction/data)

따라서 '-1'값을 NaN으로 변환하여 처리하려고 한다.

In [ ]:
train_copy = train
train_copy = train_copy.replace(-1, np.NaN)

이번에는 Kaggler [Alekse Bilogur](https://www.kaggle.com/residentmario)의 'Missingno' 패키지를 활용하여 결측치를 체크한다.

In [ ]:
import missingno as msno
msno.matrix(df=train_copy.iloc[:, 2:39], figsize=(20,14), color=(0.42, 0.1, 0.05))

그래프를 보면 알 수 있듯이, 빈 공간이 많은 부분일수록 결측치가 많은 것을 알 수 있다.

missingno plot은 각 40개의 변수들이 한 plot에 들어오는 것이 가장 자연스럽다. <br>
따라서 몇 개의 column이 누락되었고 그 중 5개의 null column이 배제되었다.

**ps_ind_05_cat | ps_reg_03 | ps_car_03_cat | ps_car_05_cat | ps_car_07_cat | ps_car_09_cat | ps_car_14** <br>
이렇게 7개의 null column을 확인할 수 있다.

그리고 **ps_car_03_cat | ps_car_05_cat | ps_car_07_cat** column은 결측 정도가 심한 것을 확인할 수 있다. 

> 따라서 '-1'을 NaN값으로 처리하는 것은 좋은 방식이 아니다.

---

## 2. 변수 검사와 필터핑 Feature Inspection and Filtering

### 타겟 변수 검사하기

타겟변수는 주로 머신러닝 지도학습에서 연관 데이터와 함께 사용된다. <br>
학습 함수의 표준화 및 데이터 예측을 위해 사용하는 예측함수가 스스로 학습을 할 때 타겟변수가 중요한 역할을 한다.

In [ ]:
data = [go.Bar(
            x = train['target'].value_counts().index.values,
            y = train['target'].value_counts().values,
            text= 'Distribution of target variable'
    )]

layout = go.Layout(
    title = 'Target variable distribution'
)

fig = go.Figure(data=data, layout=layout)

py.iplot(fig, filename='basic-bar')

타겟변수의 불균형이 너무 심하다...! 확인 해두기

### 데이터 타입 확인하기

In [ ]:
Counter(train.dtypes.values)

정수형 변수가 49개, 실수형 변수가 10개로 카운트된다.

또 다른 점은 Porto Seguro는 '_bin', '_cat', '_reg'와 같은 접미사를 데이터 헤더에 약어로 사용하는 것이다. <br>****
이런 약어들은 대략적인 설명을 담고 있다 (_bin은 이진 변수, _cat은 범주형 변수 등으로)

In [ ]:
train_float = train.select_dtypes(include=['float64'])
train_int = train.select_dtypes(include=['int64'])

In [ ]:
train_float

### Correlation plots

In [ ]:
# 실수형 변수의 상관관계 heatmap
colormap = plt.cm.magma
plt.figure(figsize=(16,12))
plt.title('Pearson correlation of continuous features', y=1.05, size=15)
sns.heatmap(train_float.corr(), linewidths=0.1, vmax=1.0, square=True,
           cmap=colormap, linecolor='white', annot=True)

양의 상관관계에 있는 변수들은 다음과 같다:

(ps_reg_01, ps_reg_02)

(ps_reg_02, ps_reg_03)

(ps_car_12, ps_car_13)

(ps_car_13, ps_car_15)

In [ ]:
# 정수형 변수의 상관관계
data = [
    go.Heatmap(
        z = train_int.corr().values,
        x = train_int.columns.values,
        y = train_int.columns.values,
        colorscale = 'Viridis',
        reversescale = False,
        opacity = 1.0 )
]

layout = go.Layout(
    title = 'Pearson Correlation of Integer-type features',
    xaxis = dict(ticks='', nticks=36),
    yaxis = dict(ticks='' ),
    width = 900, height = 700)

fig = go.Figure(data=data, layout=layout)
py.iplot(fig, filename='labelled-heatmap')

### 상호정보량 Mutual Information plots

상호정보량은 타겟변수와 그에 상응하는 변수들의 상호 관계를 확인할 수 있는 유용한 기법 중 하나이다. <br>
sklearn의 mutual_info_classif 방법을 사용하여 두가지 랜덤 변수의 의존성을 측정한다. <br>
이는 타겟 변수가 다른 변수들 사이에서 얼마나 많은 정보를 가지고 있는 지 확인할 수 있다.

In [ ]:
mf = mutual_info_classif(train_float.values, train.target.values, n_neighbors=3, random_state=17)
print(mf)

### 이진 변수 검사하기

이진 변수만을 포함하는 column에 대해서 조사하기

In [ ]:
bin_col = [col for col in train.columns if '_bin' in col]
zero_list = []
one_list = []
for col in bin_col:
    zero_list.append((train[col]==0).sum())
    one_list.append((train[col]==1).sum())

In [ ]:
trace1 = go.Bar(
    x = bin_col,
    y = zero_list,
    name = 'Zero Count'
)

trace2 = go.Bar(
    x = bin_col,
    y = one_list,
    name = 'One Count'
)

data = [trace1, trace2]
layout = go.Layout(
    barmode = 'stack',
    title = 'Count of 1 and 0 in binary variables'
)

fig = go.Figure(data=data, layout=layout)
py.iplot(fig, filename='stacked-bar')

ps_ind_10_bin, ps_ind_11_bin, ps_ind_12_bin, ps_ind_13_bin <br>
이 4가지 변수는 zero count가 대부분의 범위를 차지하고 있다.

---

## 3. 머신러닝을 통한 변수 중요도 순위 매기기

### Random Forest를 통해 알아보는 변수 중요도 

In [ ]:
from sklearn.ensemble import RandomForestClassifier
rf = RandomForestClassifier(n_estimators=150, max_depth=8, min_samples_leaf=4, max_features=0.2, n_jobs=-1, random_state=0)
rf.fit(train.drop(['id', 'target'], axis=1), train.target)
features = train.drop(['id', 'target'], axis=1).columns.values
print("----- Training Done -----")

In [ ]:
# scatter plot
trace = go.Scatter(
    y = rf.feature_importances_,
    x = features,
    mode = 'markers',
    marker = dict(
        sizemode = 'diameter',
        sizeref = 1,
        size = 13,
        color = rf.feature_importances_,
        colorscale = 'Portland',
        showscale = True
    ),
    text = features
)
data = [trace]

layout = go.Layout(
    autosize = True,
    title = 'Random Forest Feature Importance',
    hovermode = 'closest',
    xaxis = dict(
        ticklen = 5,
        showgrid = False,
        zeroline = False,
        showline = False
    ),
    yaxis = dict(
        title = 'Feature Importance',
        showgrid = False,
        zeroline = False,
        ticklen = 5,
        gridwidth = 2
    ),
    showlegend = False
)

fig = go.Figure(data=data, layout=layout)
py.iplot(fig, filename='scatter2010')

이와 더불어, 모든 변수들을 중요도 순으로 내림차순 정렬하여 bar plot으로 나타낼 수 있다.

In [ ]:
x, y = (list(x) for x in zip(*sorted(zip(rf.feature_importances_, features),
                                    reverse = False)))

trace2 = go.Bar(
    x = x,
    y = y,
    marker = dict(
        color = x,
        colorscale = 'Viridis',
        reversescale = True
    ),
    name = 'Random Forest Feature Importances',
    orientation = 'h',
)

layout = dict(
    title = 'Barplot of Feature Importances',
    width = 900, height = 1500,
    yaxis = dict(
        showgrid = False,
        showline = False,
        showticklabels = True,
            domain = [0, 0.85],
    ))

fig1 = go.Figure(data=[trace2])
fig1['layout'].update(layout)
py.iplot(fig1, filename='plots')

### Decision Tree Visualisation

In [ ]:
from sklearn import tree
from IPython.display import Image as PImage
from subprocess import check_call
from PIL import Image, ImageDraw, ImageFont
import re

decision_tree = tree.DecisionTreeClassifier(max_depth=3)
decision_tree.fit(train.drop(['id', 'target'], axis=1), train.target)

# 훈련된 모델을 .dot 파일로 변환하기
with open('tree1.dot', 'w') as f:
    f = tree.export_graphviz(decision_tree,
                            out_file = f,
                            max_depth = 4,
                            impurity = False,
                            feature_names = train.drop(['id', 'target'], axis=1).columns.values,
                            class_names = ['No', 'Yes'],
                            rounded = True,
                            filled = True)
    
# .dot 파일을 .png 파일로 변환하여 웹 노트북에서 볼 수 있게 하기
check_call(['dot', '-Tpng', 'tree1.dot', '-o', 'tree1.png'])

# Annoting chart with PIL
img = Image.open('tree1.png')
draw = ImageDraw.Draw(img)
img.save('sample-out.png')
PImage('sample-out.png')

### Gradient Boosting 모델을 통해 알아보는 변수 중요도

In [ ]:
from sklearn.ensemble import GradientBoostingClassifier
gb = GradientBoostingClassifier(n_estimators=100, max_depth=3, min_samples_leaf=4, max_features=0.2, random_state=0)
gb.fit(train.drop(['id', 'target'], axis=1), train.target)
features = train.drop(['id', 'target'], axis=1).columns.values
print('----- Training Done -----')

In [ ]:
# Scatter plot 
trace = go.Scatter(
    y = gb.feature_importances_,
    x = features,
    mode='markers',
    marker=dict(
        sizemode = 'diameter',
        sizeref = 1,
        size = 13,
        #size= rf.feature_importances_,
        #color = np.random.randn(500), #set color equal to a variable
        color = gb.feature_importances_,
        colorscale='Portland',
        showscale=True
    ),
    text = features
)
data = [trace]

layout= go.Layout(
    autosize= True,
    title= 'Gradient Boosting Machine Feature Importance',
    hovermode= 'closest',
     xaxis= dict(
         ticklen= 5,
         showgrid=False,
        zeroline=False,
        showline=False
     ),
    yaxis=dict(
        title= 'Feature Importance',
        showgrid=False,
        zeroline=False,
        ticklen= 5,
        gridwidth= 2
    ),
    showlegend= False
)
fig = go.Figure(data=data, layout=layout)
py.iplot(fig,filename='scatter2010')

In [ ]:
x, y = (list(x) for x in zip(*sorted(zip(gb.feature_importances_, features), 
                                                            reverse = False)))
trace2 = go.Bar(
    x=x ,
    y=y,
    marker=dict(
        color=x,
        colorscale = 'Viridis',
        reversescale = True
    ),
    name='Gradient Boosting Classifer Feature importance',
    orientation='h',
)

layout = dict(
    title='Barplot of Feature importances',
     width = 900, height = 2000,
    yaxis=dict(
        showgrid=False,
        showline=False,
        showticklabels=True,
    ))

fig1 = go.Figure(data=[trace2])
fig1['layout'].update(layout)
py.iplot(fig1, filename='plots')

> Observations: <br>
RandomForest과 Gradient Boosting model 둘다 제일 중요한 변수로 ps_car_13을 꼽았다.


## Conclusion

Porto Seguro 데이터셋을 가지고 결측치와 데이터 품질 검사를 진행하였고, <br>
변수들간의 선형 상관관계를 조사하며, Random Forest와 Gradient Boosting model을 통해 변수 중요도를 판단해보았다.

이를 통해 중요한 변수를 선별하는 작업이 가능하였다.